In [38]:
from frust.stepper import Stepper
from pathlib import Path
from tooltoad.chemutils import xyz2mol

In [39]:
structures_path = Path("../structures/irc_tests/irc_new_ts/")

mols = {}

for f in structures_path.rglob("*.xyz"):
    with open(f, "r") as file:
        xyz_block = file.read()
        mol = xyz2mol(xyz_block)
        mols[f.stem] = (mol, [0])
mols        

{'ts3': (<rdkit.Chem.rdchem.Mol at 0x7f36713d1bd0>, [0]),
 'ts4': (<rdkit.Chem.rdchem.Mol at 0x7f36713d3c30>, [0])}

In [40]:
step = Stepper(list(mols.keys()), step_type="IRC", save_output_dir=False)
df0 = step.build_initial_df(mols)
df0

2025-07-15 15:23:59 INFO  frust.stepper: Working dir: .


,custom_name,ligand_name,rpos,constraint_atoms,cid,smiles,atoms,coords_embedded,energy_uff
0,ts3,ts3,<NA>,None,0,None,"[C, C, C, C, C, C, H, H, H, H, B, H, C, C, C, ...","[(-2.056478, -2.08167, 0.887745), (-0.715417, ...",None
1,ts4,ts4,<NA>,None,0,None,"[C, C, C, C, C, C, H, H, H, H, N, B, H, H, H, ...","[(-2.234946, -1.448244, 0.976903), (-1.070948,...",None


In [41]:
dfs = []
for i in range(len(df0)):
    df_tmp = df0.iloc[[i]]
    dfs.append(df_tmp)

def run_irc(df, debug=False, n_cores=12, mem_gb=30):
    from pathlib import Path
    name = df["custom_name"].iloc[0]

    results_dir = Path("results_irc")
    results_dir.mkdir(exist_ok=True)

    step = Stepper([name],
                step_type="IRC",
                debug=debug,
                save_output_dir=False,
                output_base=str(results_dir),
                n_cores=n_cores,
                memory_gb=mem_gb)
    
    detailed_inp = """%IRC
    MAXITER 100
    END
    """
    options = {
        "wB97X-D3" : None,
        "6-31G**"  : None,
        "TightSCF" : None,
        "Freq"     : None,
        "NoSym"    : None,
        "IRC"      : None,
    }

    df1 = step.orca(df, f"DFT-IRC", options, detailed_inp, save_files=["orca.out", "*.xyz"])
    df1.to_parquet(f"{results_dir}/df_irc_{name}.parquet")

In [42]:
import submitit

DEBUG           = False
N_CORES         = 8
MEM_GB          = 30
TIMEOUT_MIN     = 14400


executor = submitit.AutoExecutor("logs/irc")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
)

for dfi in dfs:
    name = "IRC_" + dfi["custom_name"].iloc[0] 
    executor.update_parameters(slurm_job_name=name)
    executor.submit(run_irc, dfi, DEBUG, N_CORES, MEM_GB)
    print(f"Submitted: {name}")

Submitted: IRC_ts3
Submitted: IRC_ts4
